# Deep Learning Training code

This notebook describes our deep learning function. It sets up the architecture, pre-processed the input data and runs the specified number of epochs.
We will describe the code step by step.

## Overview
The deep learning architecture is built using `Keras`, a model-level library that provides high-level building blocks for developing deep learning models. It provides a user-friendly interface to low-level tensor manipulation libraries that handle the computationally intensive tensor operations involved in model fitting (e.g. `Tensorflow`, `Theano`, `CNTK`). For this work, we used `Keras` interfaced with `Tensorflow` as the backend computational engine. 

### 1. Importing Libraries 
We begin by importing required libraries: 

In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l1, l2, l1_l2
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
import os, sys, time, pickle, copy

### 2. Data formatting
Define function that converts input to require formats for analysis

In [ ]:
def prepare_data(x, e, t):
    return (x.astype("float32"), e.astype("int32"), t.astype("float32"))

### Define `DL_single_run` function
We define function `DL_single_run` below. The first 2 function arguments are variables `xtr` and `ytr`, which represent the training data. Variable `xtr` is an $n \times p$ matrix (where $n$ is sample size and $p$ is number of variables [motion descriptors, in our case]). Variable `ytr` contains the survival outcomes for the training data, it is an $n \times 2$ matrix where the first column is the censoring status and the second is survival/censoring time.
The `DL_single_run` function also has arguments representing network parameters: `units1` (hidden layer size), `units2` (latent code size), `dro` (dropout), `lr` (learning rate), `l1r` (L1 penalty for activity regularizer), and `alpha` (tradeoff parameter). The final arguments are `batchsize` (batch size) and `numepochs` (number of epochs).

The first steps of the function format the data into the correct numeric types, and sort into minibatches:

In [ ]:
def DL_single_run(xtr, ytr, units1, units2, dro, lr, l1r, alpha, batchsize, numepochs):
    #Data preparation: create X, E and TM where X=input vector, E=censoring status and T=survival time. Apply formatting
    X_tr, E_tr, TM_tr = prepare_data(xtr, ytr[:,0,np.newaxis], ytr[:,1])

    #Arrange data into minibatches (based on specified batch size), and within each minibatch, sort in descending order of survival/censoring time (see explanation of Cox PH loss function definition)
    X_tr, E_tr, TM_tr, _ = sort4minibatches(X_tr, E_tr, TM_tr, batchsize)

### 3. Model  Architecture
Still within the `DL_single_run` function definition, we set up Denoising autoencoder model in *Keras*.

In [ ]:
    #before defining network architecture, clear current computational graph (if one exists), and specify input dimensionality)
    K.clear_session()
    inpshape = xtr.shape[1]
    
    #Define Network Architecture
    inputvec= Input(shape=(inpshape,))
    x       = Dropout(dro, input_shape=(inpshape,))(inputvec)
    x       = Dense(units=int(units1), activation='relu', activity_regularizer=l1(10**l1r))(x)
    encoded = Dense(units=int(units2), activation='relu', name='encoded')(x)
    riskpred= Dense(units=1,  activation='linear', name='predicted_risk')(encoded)
    z       = Dense(units=int(units1),  activation='relu')(encoded)
    decoded = Dense(units=inpshape, activation='linear', name='decoded')(z)

    model = Model(inputs=inputvec, outputs=[decoded,riskpred])
    model.summary()

### 4. Define Network prediction loss function
We use the negative Cox Proportional Hazards Partial Likelihood function: $$L_{s} =  -\sum_{i=1}^{n} \delta_{i} \bigg\{\bf{W}'\phi(\bf{x}_i)  - log \sum_{j \in R(t_i)} e^{\bf{W}'\phi(\bf{x}_j)} \bigg\}$$
for the survival prediction component of our network. This loss function is not one of the standard losses included in *Keras*. However, *Keras* allows creation of user-defined custom loss functions written using symbolic Keras backend operations, which operate on tensor objects (as opposed to python primitives). We show our user-defined function below.

In [ ]:
    def _negative_log_likelihood(E, risk):  
        hazard_ratio = K.exp(risk)
        log_risk = K.log(K.cumsum(hazard_ratio))
        uncensored_likelihood = risk - log_risk
        censored_likelihood = uncensored_likelihood * E
        neg_likelihood = -K.sum(censored_likelihood)
        return neg_likelihood

In the function, argument `E` represents censoring status ($\delta_{i}$) and `risk` ($\bf{W}'\phi(\bf{x}_i)$) represents the network output, i.e. the log hazard ratio. Before being passed to the `_negative_log_likelihood` function below, the data for the `E` and `risk` arguments must be sorted in order of descending survival time. This is because of the use of the `cumsum` (cumulative sum) Keras backend function on *Line 3* of the definition, which sums hazard ratios over the risk sets of each patient (see second term in above equation).

### 4b. Arrange data into minibatches sorted by descending time
We use the function below, with arguments `xvals` (input vector of motion descriptors), `evals` (censoring statuses), `tvals` (survival/censoring time [in days]), `batchsize` (desired batch size)

In [ ]:
def sort4minibatches(xvals, evals, tvals, batchsize):
    ntot = len(xvals)
    indices = np.arange(ntot)
    np.random.shuffle(indices)
    start_idx=0
    esall = []
    for end_idx in list(range(batchsize, batchsize*(ntot//batchsize)+1, batchsize))+[ntot]:
        excerpt = indices[start_idx:end_idx]
        sort_idx = np.argsort(tvals[excerpt])[::-1]
        es = excerpt[sort_idx]
        esall += list(es)
        start_idx = end_idx
    return (xvals[esall], evals[esall], tvals[esall], esall)

### 5. Compile and Run Network
Still within the `DL_single_run` function definition, we compile the model and run. For compilation, we specify our 2 losses: reconstruction loss (`mean_squared_error`) and our custom Cox prediction loss (`_negative_log_likelihood`). Their corresponding weights are defined based on the `alpha` parameter.
Note that for running the model (`model.fit`), we set `shuffle=False` to ensure that the predefined batch size order (see `sort4minibatches`) is preserved from epoch to epoch. 
The last line (`return mlog`) of the `DL_single_run` function definition returns the log file produced by `model.fit`. This contains epoch history (losses at each epoch), the fitted model as a `Keras` model object (architecture, weights, etc.), and other information about the fit.

In [ ]:
    #Model compilation
    optimdef = Adam(lr = lr)
    model.compile(loss=[keras.losses.mean_squared_error, _negative_log_likelihood], loss_weights=[alpha,1-alpha], optimizer=optimdef, metrics={'decoded':keras.metrics.mean_squared_error})
    
    #Run model
    mlog = model.fit(X_tr, [X_tr,E_tr], batch_size=batchsize, epochs=numepochs, shuffle=False, verbose=1)

    K.clear_session()
    return mlog